In [5]:
import pandas as pd
from sqlalchemy import create_engine
from password import password

In [6]:
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'churn_pred'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
connection = engine.connect()

In [7]:
df = pd.read_sql("SELECT * FROM fact_churn", connection)
df.head()

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,...,state_WI,state_WV,state_WY,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,international_plan_no,international_plan_yes,voice_mail_plan_no,voice_mail_plan_yes
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,...,0,0,0,0,1,0,1,0,0,1
1,118,0,223.4,98,37.98,220.6,101,18.75,203.9,118,...,0,0,0,0,0,1,0,1,1,0
2,62,0,120.7,70,20.52,307.2,76,26.11,203.0,99,...,0,0,0,0,1,0,1,0,1,0
3,93,0,190.7,114,32.42,218.2,111,18.55,129.6,121,...,0,0,0,0,0,1,1,0,1,0
4,174,0,124.3,76,21.13,277.1,112,23.55,250.7,115,...,0,0,0,0,1,0,1,0,1,0


In [8]:
# Split the data into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(['churn'], axis=1)
y = df['churn'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.33)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X.sample(5)

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,...,state_WI,state_WV,state_WY,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,international_plan_no,international_plan_yes,voice_mail_plan_no,voice_mail_plan_yes
2033,103,0,173.5,83,29.50,244.3,65,20.77,221.6,66,...,0,0,0,0,1,0,1,0,1,0
1877,7,30,221.4,114,37.64,165.8,116,14.09,247.0,105,...,0,0,0,1,0,0,1,0,0,1
344,93,32,116.9,120,19.87,232.4,97,19.75,127.7,112,...,0,0,0,0,1,0,1,0,0,1
4183,124,0,133.2,80,22.64,144.2,98,12.26,261.0,102,...,0,0,0,0,0,1,1,0,1,0
463,103,37,153.5,78,26.10,241.9,108,20.56,244.7,110,...,0,0,0,0,1,0,1,0,0,1


In [9]:
# Train a Logistic Regression model print the model score
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(max_iter=10000)
log.fit(X_train_scaled, y_train)

print(f"Training Data Score: {log.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {log.score(X_test_scaled, y_test)}")

Training Data Score: 0.8397014925373134
Testing Data Score: 0.8090909090909091
